In [56]:
import pandas as pd
import numpy as np 
import os

print(os.getcwd())

""" p01 = pd.read_parquet("../data/updated/p01.parquet")
p02 = pd.read_parquet("../data/updated/p02.parquet")
p03 = pd.read_parquet("../data/updated/p03.parquet")
p04 = pd.read_parquet("../data/updated/p04.parquet")
p05 = pd.read_parquet("../data/updated/p05.parquet")

componenti = pd.read_parquet("../data/updated/components.parquet")
missions = pd.read_parquet("../data/updated/missions.parquet")
comuni = pd.read_parquet("../data/updated/comuni.parquet")

indicatori_sdg = pd.read_parquet("../data/updated/indicators_sdg.parquet")
indicatori_comuni=pd.read_csv("../data/updated/PNRR_Indicatori_Comuni-Universo_REGIS_v2.csv", sep=";", encoding='latin-1')
#indicatori_comuni = pd.read_csv("../data/updated/PNRR_Indicatori_Comuni-Universo_REGIS_v2.csv",sep=",")
"""
appalti_2024=pd.read_excel("/Users/venere/Desktop/period/data-prep-pnrr/data/updated/APPALTI_2024_PNRR_PNC_Full 2.xlsx", header=0)

#appalti_2024=pd.read_csv("/Users/venere/Desktop/period/data-prep-pnrr/data/updated/APPALTI_2024_PNRR_PNC_csv.csv", sep=",")

/Users/venere/Desktop/period/data-prep-pnrr/notebooks/data_prep_old/current_app


In [98]:
appalti_2024.columns

Index(['cig', 'cf_sa', 'Denominazione SA', 'data_pubblicazione',
       'importo_lotto', 'oggetto_lotto', 'tipo_procedura', 'flag_pnrr_pnc',
       'flag_quote_deroga', 'quota_femminile', 'quota_giovani',
       'cod_mot_deroga', 'motivo_deroga', 'flag_misure_premiali',
       'codice_mis_prem', 'misura_premiale'],
      dtype='object')

In [ ]:
appalti_2024

In [100]:
appalti_2024
appalti_2024 = appalti_2024[['cig','data_pubblicazione',
       'importo_lotto','flag_quote_deroga', 'quota_femminile', 'quota_giovani',
       'cod_mot_deroga', 'motivo_deroga', 'flag_misure_premiali',
       'codice_mis_prem', 'misura_premiale']]

In [101]:
premialita_24 = appalti_2024[appalti_2024["flag_misure_premiali"]=='[]']
x = appalti_2024.shape
y = premialita_24.shape
x = x[0]
y = y[0]
percentuale = y/x*100
print(x, y, percentuale)


51395 1852 3.6034633719233393


In [102]:
import ast
premialita_24['codice_mis_prem'] = premialita_24['codice_mis_prem'].apply(ast.literal_eval)
premialita_24['misura_premiale'] = premialita_24['misura_premiale'].apply(ast.literal_eval)

/var/folders/zx/2_t1dk8d70ld1mtph0ys7mt40000gn/T/ipykernel_29095/3802793326.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  premialita_24['codice_mis_prem'] = premialita_24['codice_mis_prem'].apply(ast.literal_eval)
/var/folders/zx/2_t1dk8d70ld1mtph0ys7mt40000gn/T/ipykernel_29095/3802793326.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  premialita_24['misura_premiale'] = premialita_24['misura_premiale'].apply(ast.literal_eval)


In [103]:
premialita_24_exploded = premialita_24.explode(['codice_mis_prem', 'misura_premiale']).reset_index(drop=True)


Deroghe e codice motivo deroga

In [104]:
print(appalti_2024.columns)


Index(['cig', 'data_pubblicazione', 'importo_lotto', 'flag_quote_deroga',
       'quota_femminile', 'quota_giovani', 'cod_mot_deroga', 'motivo_deroga',
       'flag_misure_premiali', 'codice_mis_prem', 'misura_premiale'],
      dtype='object')


In [105]:
deroghe_24 = appalti_2024[appalti_2024["flag_quote_deroga"].fillna('').isin(['Q', 'N'])]

deroghe_24.shape

(8061, 11)

In [106]:
def safe_eval(val):
    if isinstance(val, str):  # Applica solo se è una stringa
        try:
            return ast.literal_eval(val)
        except (ValueError, SyntaxError):
            return val  # Ritorna il valore originale se non è una lista valida
    return val  # Se è un numero o NaN, lascia invariato

deroghe_24['cod_mot_deroga'] = deroghe_24['cod_mot_deroga'].apply(safe_eval)
deroghe_24['motivo_deroga'] = deroghe_24['motivo_deroga'].apply(safe_eval)

deroghe_24_exploded = deroghe_24.explode(["cod_mot_deroga", "motivo_deroga"]).reset_index(drop=True)


/var/folders/zx/2_t1dk8d70ld1mtph0ys7mt40000gn/T/ipykernel_29095/2803642113.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  deroghe_24['cod_mot_deroga'] = deroghe_24['cod_mot_deroga'].apply(safe_eval)
/var/folders/zx/2_t1dk8d70ld1mtph0ys7mt40000gn/T/ipykernel_29095/2803642113.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  deroghe_24['motivo_deroga'] = deroghe_24['motivo_deroga'].apply(safe_eval)


In [107]:
deroghe_24_exploded.shape

(8168, 11)

*Analisi premialità*

Nella scorsa versione erano 11 misure premiali, ora risultano 13, necessario analizzarle e aggiornare la mappatura

In [108]:
premialita_24_exploded["misura_premiale"].unique()

array(['UTILIZZI O SI IMPEGNI A UTILIZZARE SPECIFICI STRUMENTI DI CONCILIAZIONE DELLE ESIGENZE DI CURA, DI VITA E DI LAVORO PER I PROPRI DIPENDENTI, NONCHÉ MODALITÀ INNOVATIVE DI ORGANIZZAZIONE DEL LAVORO',
       "SI IMPEGNI AD ASSUMERE, OLTRE ALLA SOGLIA MINIMA PERCENTUALE PREVISTA COME REQUISITO DI PARTECIPAZIONE, DONNE PER L'ESECUZIONE DEL CONTRATTO O PER LA REALIZZAZIONE DI ATTIVITÀ AD ESSO CONNESSE O STRUMENTALI",
       "SI IMPEGNI AD ASSUMERE, OLTRE ALLA SOGLIA MINIMA PERCENTUALE PREVISTA COME REQUISITO DI PARTECIPAZIONE GIOVANI DI ETÀ INFERIORE A TRENTASEI ANNI PER L'ESECUZIONE DEL CONTRATTO O PER LA REALIZZAZIONE DI ATTIVITÀ AD ESSO CONNESSE O STRUMENTALI",
       "ABBIA, NELL'ULTIMO TRIENNIO, RISPETTATO GLI OBBLIGHI DI CUI ALLA LEGGE 12 MARZO 1999, N. 68 IN MATERIA DI LAVORO DELLE PERSONE DISABILI",
       'ALTRO CRITERIO DI RESPONSABILITÀ SOCIALE NEI CONFRONTI DEI DIVARI DI GENERE',
       'ALTRO CRITERIO DI RESPONSABILITÀ SOCIALE NEI CONFRONTI DEI DIVARI INTERGENERAZIONALI

In [109]:
premialita_24_exploded.to_csv("premialita_24_exploded.csv", index = False, sep=';')
#cod_mis_premiale
premialita_24_exploded = premialita_24_exploded.rename(columns={'codice_mis_prem': 'cod_mis_premiale'})

Riapplichiamo la mappatura fatta in precedenza per le misure premiali

In [ ]:
premialita_24_exploded

In [ ]:
def conditions(s):
    if s['cod_mis_premiale'] in [1,2,9]:
        return "GENERALE"
    elif s['cod_mis_premiale'] in [3,8,12]:
        return "DISABILI"
    elif s['cod_mis_premiale'] in [4,6,10]:
        return "GENERE"
    elif s['cod_mis_premiale'] in [5,7,11]:
        return "ETÀ"
    else:
        return 0
    
premialita_24_exploded['CATEGORIA'] = premialita_24_exploded.apply(conditions,axis=1)
premialita_24_exploded

Motivo deroga è coerente.
Creiamo il dataset con premialità e deroghe.

In [113]:
appalti_2024 = appalti_2024[['cig','data_pubblicazione',
       'importo_lotto']]

In [ ]:
leftpremialita = appalti_2024.merge(premialita_24_exploded, on='cig',how='left')
leftpremialita
leftpremialita = leftpremialita[['cig','data_pubblicazione_x', 'importo_lotto_x', 'flag_misure_premiali',	'cod_mis_premiale',	'misura_premiale',	'CATEGORIA']]
leftpremialita = leftpremialita.rename(columns={'importo_lotto_x': 'importo_lotto'})
leftpremialita = leftpremialita.rename(columns={'data_pubblicazione_x': 'data_pubblicazione'})
leftpremialita

In [128]:
leftquote = leftpremialita.merge(deroghe_24_exploded, on='cig',how='left')
leftquote
leftquote = leftquote[['cig','data_pubblicazione_x', 'importo_lotto_x', 'flag_misure_premiali_x',	'cod_mis_premiale',	'misura_premiale_x',	'CATEGORIA','flag_quote_deroga', 'quota_femminile',	'quota_giovani', 'cod_mot_deroga',	'motivo_deroga']]
leftquote = leftquote.rename(columns={'data_pubblicazione_x': 'data_pubblicazione'})
leftquote = leftquote.rename(columns={'importo_lotto_x': 'importo_lotto'})
leftquote = leftquote.rename(columns={'flag_misure_premiali_x': 'flag_misure_premiali'})
leftquote = leftquote.rename(columns={'misura_premiale_x': 'misura_premiale'})

leftquote

,cig,data_pubblicazione,importo_lotto,flag_misure_premiali,cod_mis_premiale,misura_premiale,CATEGORIA,flag_quote_deroga,quota_femminile,quota_giovani,cod_mot_deroga,motivo_deroga
0,A01C3C50B3,2024-01-01,684237.20,[],2,UTILIZZI O SI IMPEGNI A UTILIZZARE SPECIFICI S...,GENERALE,Q,15.0,30.0,2,MERCATO DI RIFERIMENTO
1,A01C3C50B3,2024-01-01,684237.20,[],2,UTILIZZI O SI IMPEGNI A UTILIZZARE SPECIFICI S...,GENERALE,Q,15.0,30.0,5,SCARSA OCCUPAZIONE FEMMINILE NEL SETTORE
2,A01C3C50B3,2024-01-01,684237.20,[],4,"SI IMPEGNI AD ASSUMERE, OLTRE ALLA SOGLIA MINI...",GENERE,Q,15.0,30.0,2,MERCATO DI RIFERIMENTO
3,A01C3C50B3,2024-01-01,684237.20,[],4,"SI IMPEGNI AD ASSUMERE, OLTRE ALLA SOGLIA MINI...",GENERE,Q,15.0,30.0,5,SCARSA OCCUPAZIONE FEMMINILE NEL SETTORE
4,A01C3C50B3,2024-01-01,684237.20,[],5,"SI IMPEGNI AD ASSUMERE, OLTRE ALLA SOGLIA MINI...",ETÀ,Q,15.0,30.0,2,MERCATO DI RIFERIMENTO
...,...,...,...,...,...,...,...,...,...,...,...,...
54811,B5133EBD2E,2024-12-31,220000.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
54812,B5137B8024,2024-12-31,91177.99,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
54813,B5138B4019,2024-12-31,292652.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
54814,B51390A70F,2024-12-31,531650.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
indicatori_comuni

print(indicatori_comuni["Descrizione Indicatore"].unique());
print(indicatori_comuni["Codice Indicatore"].unique())

def conditions_persone(s):
    if s['Codice Indicatore'] in [
        "C10.A",
        "C10.B",
        "C10.C",
        "C10.D",
        "C10.E",
        "C10.F",
        "C10.G",
        "C10.H",
        "C10IA",
        "C10IB",
        "C10IC",
        "C10ID",
        "C10IE",
        "C10IF",
        "C10IG",
        "C10IH",
        "C11.A",
        "C11.B",
        "C11.C",
        "C11.D",
        "C11.E",
        "C11.F",
        "C11.G",
        "C11.H",
        "C12",
        "C14.F",
        "C14.M",
        "C4",
        "C7",
        "C8.F",
        "C8.M"]:
        return "1"
    else:
        return 0
    
indicatori_comuni['FLG_INDICATORI_PERSONE'] = indicatori_comuni.apply(conditions_persone, axis=1)
indicatori_comuni

indicatori_comuni

print(indicatori_comuni["Descrizione Indicatore"].unique());
print(indicatori_comuni["Codice Indicatore"].unique())

def conditions_persone(s):
    if s['Codice Indicatore'] in [
        "C10.A",
        "C10.B",
        "C10.C",
        "C10.D",
        "C10.E",
        "C10.F",
        "C10.G",
        "C10.H",
        "C10IA",
        "C10IB",
        "C10IC",
        "C10ID",
        "C10IE",
        "C10IF",
        "C10IG",
        "C10IH",
        "C11.A",
        "C11.B",
        "C11.C",
        "C11.D",
        "C11.E",
        "C11.F",
        "C11.G",
        "C11.H",
        "C12",
        "C14.F",
        "C14.M",
        "C4",
        "C7",
        "C8.F",
        "C8.M"]:
        return "1"
    else:
        return 0
    
indicatori_comuni['FLG_INDICATORI_PERSONE'] = indicatori_comuni.apply(conditions_persone, axis=1)
indicatori_comuni

def conditions_persone(s):
    if s['Codice Indicatore'] in [
        "C10.A",
        "C10.B",
        "C10.C",
        "C10.D",
        "C10.E",
        "C10.F",
        "C10.G",
        "C10.H",
        "C10IA",
        "C10IB",
        "C10IC",
        "C10ID",
        "C10IE",
        "C10IF",
        "C10IG",
        "C10IH",
        "C11.A",
        "C11.B",
        "C11.C",
        "C11.D",
        "C11.E",
        "C11.F",
        "C11.G",
        "C11.H",
        "C12",
        "C14.F",
        "C14.M",
        "C4",
        "C7",
        "C8.F",
        "C8.M"]:
        return "1"
    else:
        return 0
    
indicatori_comuni['FLG_INDICATORI_PERSONE'] = indicatori_comuni.apply(conditions_persone, axis=1)
indicatori_comuni


def conditions_genere(s):
    if s['Codice Indicatore'] in [
        "C10.E",
        "C10.F",
        "C10.G",
        "C10.H",
        "C10IE",
        "C10IF",
        "C10IG",
        "C10IH",
        "C11.E",
        "C11.F",
        "C11.G",
        "C11.H",
        "C14.F",
         "C8.F"]:
        return "1"
    else:
        return 0

indicatori_comuni['FLG_INDICATORI_GENERE'] = indicatori_comuni.apply(conditions_genere, axis=1) 
indicatori_comuni
indicatori_comuni.to_csv("indicatori_comuni.csv", index = False, sep=';')

In [ ]:
p05 = p05.rename(columns={'cig':'CIG'})
p03 = p03.rename(columns={'cig':'CIG'})
p04 = p04.rename(columns={'cig':'CIG'})
def conditions(s):
    if s['cod_mis_premiale'] in [1,2,9]:
        return "GENERALE"
    elif s['cod_mis_premiale'] in [3,8,12]:
        return "DISABILI"
    elif s['cod_mis_premiale'] in [4,6,10]:
        return "GENERE"
    elif s['cod_mis_premiale'] in [5,7,11]:
        return "ETÀ"
    else:
        return 0
    
p03['CATEGORIA'] = p03.apply(conditions,axis=1)

# left join p5 e p3
leftnew_3 = p05.merge(p03, on='CIG',how='left')
leftnew_3 = leftnew_3.drop_duplicates()
print(leftnew_3.columns)
print(leftnew_3.shape)
# left join il risultato con p4
leftnew_34 = leftnew_3.merge(p04, on='CIG', how='left')
leftnew_34 = leftnew_34.drop_duplicates()
print(leftnew_34.columns)
print(leftnew_34.shape)
leftnew_34.to_csv("leftnew_34.csv", index = False, sep=';')